In [1]:
import pandas as pd 
import numpy as np 
import pickle 
import torch as t
from sklearn.model_selection import train_test_split

In [4]:
data = t.load('results/llama/logistic_regression/directions_residual')

tensor([-5.4883, -6.7148, -4.4922,  ..., -6.1328, -6.7227, -6.4883],
       device='cuda:0', dtype=torch.float16, requires_grad=True)

In [74]:
import torch

def to_numpy(item):
        if isinstance(item, torch.Tensor):
            return item.detach().cpu().numpy()
        elif isinstance(item, np.ndarray):
            return item
        elif isinstance(item, list):
            return [to_numpy(x) for x in item]
        else:  # scalar
            return item

In [75]:
data = to_numpy(data)

In [76]:
data

[array([-5.49 , -6.715, -4.492, ..., -6.133, -6.723, -6.49 ], dtype=float16),
 array([-4.918,  3.424, -5.633, ..., -4.402, -5.914, -6.23 ], dtype=float16),
 array([-4.387,  4.793, -6.39 , ...,  4.758, -5.426, -6.22 ], dtype=float16),
 array([-6.86 ,  3.734, -4.273, ...,  3.83 , -5.855, -6.305], dtype=float16),
 array([-5.45   , -6.074  ,  1.013  , ..., -0.10614, -1.011  , -2.691  ],
       dtype=float16),
 array([-3.787, -3.908,  0.759, ...,  4.61 ,  1.56 , -3.348], dtype=float16),
 array([ 4.312, -5.56 ,  2.186, ...,  4.535,  0.781, -2.72 ], dtype=float16),
 array([ 1.907  , -4.773  , -0.12494, ...,  4.375  , -5.957  , -1.106  ],
       dtype=float16),
 array([ 1.29 , -5.348,  2.643, ...,  4.69 , -4.51 , -2.762], dtype=float16),
 array([ 4.8   , -5.332 ,  4.293 , ...,  4.39  , -2.73  , -0.4587],
       dtype=float16),
 array([ 3.922 , -5.08  ,  4.355 , ...,  2.18  , -0.4944, -0.565 ],
       dtype=float16),
 array([ 5.207  , -3.578  ,  4.312  , ..., -0.4556 , -0.01031, -1.173  ],
    

In [37]:
data_np = np.array([x.detach().cpu().numpy() if isinstance(x, t.Tensor) else x for x in data])

TypeError: can't convert cuda:0 device type tensor to numpy. Use Tensor.cpu() to copy the tensor to host memory first.

In [33]:
data_np

array([[0.49798966, 0.49798966, 0.49798966, ..., 0.49798966, 0.49798966,
        0.49798966],
       [0.49052269, 0.49798966, 0.49798966, ..., 0.49798966, 0.5066054 ,
        0.50143596],
       [0.49798966, 0.49798966, 0.49798966, ..., 0.49798966, 0.49798966,
        0.49798966],
       ...,
       [0.49798966, 0.49798966, 0.47501436, ..., 0.69557725, 0.61458932,
        0.62377944],
       [0.67662263, 0.68064331, 0.65422171, ..., 0.62377944, 0.65364733,
        0.74325101],
       [0.65709362, 0.70476738, 0.74210224, ..., 0.75186674, 0.63871338,
        0.74784607]])

In [11]:
# Logical - easy
with open(f'data/datasets/coherence/curated_dataset_full.pkl', 'rb') as file:
    curated_dataset = pickle.load(file)
with open(f'data/datasets/coherence/conj_dataset.pkl', 'rb') as file:
    conj_dataset = pickle.load(file)
with open(f'data/datasets/coherence/neg_dataset.pkl', 'rb') as file:
    neg_dataset = pickle.load(file)

In [12]:
neg_dataset['filename'].unique()

array(['neg_common_claim_true_false.csv', 'neg_companies_true_false.csv',
       'neg_counterfact_true_false.csv'], dtype=object)

In [56]:
def get_data_split(task: str, curated_dataset, probe_config, other_dataset=None, cutoff=1500):


    if task in ['negation', 'disjunction', 'conjunction']:
        # In this case curated dataset is the logical dataset + the remainder
        remainder = curated_dataset[~curated_dataset['filename'].isin(['common_claim_true_false.csv', 'companies_true_false.csv', 'counterfact_true_false.csv'])]
        curated_dataset = pd.concat([remainder, other_dataset])
        train_set, test_set = train_test_split(curated_dataset, test_size=0.2, random_state=42, stratify=curated_dataset['filename'])
        test_df = test_set.dropna()
        
    elif task == 'inference':
        remainder = curated_dataset[~curated_dataset['filename'].isin(['common_claim_true_false.csv', 'companies_true_false.csv', 'counterfact_true_false.csv',
                                                                      'cities.csv', 'cities_cities_conj.csv', 'cities_cities_disj.csv', 'neg_cities.csv'
                                                                      ])]
        curated_dataset = pd.concat([remainder, other_dataset])
        train_set, test_set = train_test_split(curated_dataset, test_size=0.2, random_state=42, stratify=curated_dataset['filename'])
        test_df = test_set.dropna()
        

    # Trim for batch size
    train_set = train_set.iloc[:-(len(train_set) % probe_config), :]
    X_clean_train = list(train_set['statement'])
    y_clean_train = list(train_set['label'])
    
    return (X_clean_train, y_clean_train, test_df)